# IEEE 802.15.4

Considering the pseudocode in the slides for a ESP32-based IoT monitoring system.
Assuming the system is operated with IEEE 802.15.4 in beacon-enabled mode (CFP only) and that the number of people present in the camera frame at any instant follows a Poisson distribution with an average rate of lambda = 0.15 persons/frame

#### 1. Compute the Probability Mass Function of the output rate of the ESP32 P(r = r_0), P(r = r_1), P(r = r_2), where r_0, r_1 and r_2 are the output rates when there are 0, 1 or more than 1 persons in the captured frame, respectively.

In [1]:
from scipy.stats import poisson
import math

# Probability Mass Function
person_rate = 0.15
P = dict()
P[0] = poisson.pmf(0, person_rate)
P[1] = poisson.pmf(1, person_rate)
P[2] = 1 - poisson.cdf(1, person_rate)

print('Using the formula for the Poisson distribution we get:')
for i in range(3):
    print(f'P(r = r_{i}) = {P[i]:.3f}')

Using the formula for the Poisson distribution we get:
P(r = r_0) = 0.861
P(r = r_1) = 0.129
P(r = r_2) = 0.010


#### 2. Based on the output rate PMF, compute a consistent slot assignment for the CFP in a monitoring system composed of 1 PAN coordinator and 3 camera nodes. Assume nominal bit rate R=250kbps, packets of L=128bytes, 1 packet fits exactly in one slot. Compute Ts (slot time), Number of slots in the CFP, Tactive, Tinactive and the duty cycle of the system.

In [2]:
# Every camera is processing a frame every 10 seconds
interval = 10

# Possible payload sizes in KB
payload = dict()
payload[0] = 1
payload[1] = 3
payload[2] = 6

# Equivalent rate in B/s
rate = dict()
for i in range(3):
    rate[i] = payload[i] * 1e3 / interval

# Number of cameras
num_cameras = 3
# Lenght of packet in B
L = 128
# Nominal bit rate in kbps
R = 250

# Slot time in ms
Ts = L * 8 / R
print(f'With the given packet lenght and nominal bit rate we get Ts = {Ts} ms.\n')

# Num of slots required by each camera in the worst case
required_slots = math.ceil(max(rate.values()) / min(rate.values()))
print(f'Every camera in the worst case requires {required_slots} slots.')

# Slots in the CFP
Ncfp = required_slots * num_cameras
print(f'Having {num_cameras} cameras the total number of slots in the CFP is {Ncfp}.\n')

# Beacon interval
BI = L / min(rate.values())
print(f'We can calculate the beacon interval as packet lenght over slowest equivalent rate and we get BI = {BI} s.')

# Active part duration
Tactive = (1 + Ncfp) * Ts
print(f'With that number of CFP slots and assuming no CAP slots we get Tactive = {Tactive} ms.')

# Inactive part duration
Tinactive = BI*1e3 - Tactive
print(f'The remaining time in the beacon interval is Tinactive = {Tinactive:.3f} ms.\n')

# Duty cycle
n = Tactive / (BI*1e3)
print(f'We can now calculate the duty cycle as active over total time and we get η = {n*100}%.')

With the given packet lenght and nominal bit rate we get Ts = 4.096 ms.

Every camera in the worst case requires 6 slots.
Having 3 cameras the total number of slots in the CFP is 18.

We can calculate the beacon interval as packet lenght over slowest equivalent rate and we get BI = 1.28 s.
With that number of CFP slots and assuming no CAP slots we get Tactive = 77.824 ms.
The remaining time in the beacon interval is Tinactive = 1202.176 ms.

We can now calculate the duty cycle as active over total time and we get η = 6.08%.


#### 3. How many additional cameras can be added to keep the duty cycle below 10%?

In [3]:
# Maximum CFP slots with 10% duty cycle
max_ncfp = (0.1 * BI * 1e3 / Ts) - 1

max_num_cameras = math.floor(max_ncfp / required_slots)
print(f'In order to keep the duty cycle below 10% we can have at most {max_num_cameras} cameras.')

max_additional_cameras = max_num_cameras - num_cameras
print(f'Having already {num_cameras} we can only add {max_additional_cameras} additional cameras.')

In order to keep the duty cycle below 10% we can have at most 5 cameras.
Having already 3 we can only add 2 additional cameras.
